In [7]:
from osgeo import gdal
import os
#归一化植被指数
def get_ndvi(path):  # 计算某一影像的ndvi值，返回二维数组
    dataset = gdal.Open(path)
    cols = dataset.RasterXSize  # 列数
    rows = dataset.RasterYSize  # 行数

    band8 = dataset.GetRasterBand(8).ReadAsArray(0, 0, cols, rows)
    band4 = dataset.GetRasterBand(4).ReadAsArray(0, 0, cols, rows)
    molecule = band8 - band4
    denominator = band8 + band4
    del dataset
    band = molecule / denominator
    band[band > 1] = 9999  # 过滤异常值
    return band


# 比值植被指数
def get_rvi(path):  # 计算某一影像的ndvi值，返回二维数组
    dataset = gdal.Open(path)
    cols = dataset.RasterXSize  # 列数
    rows = dataset.RasterYSize  # 行数

    band8 = dataset.GetRasterBand(8).ReadAsArray(0, 0, cols, rows)
    band4 = dataset.GetRasterBand(4).ReadAsArray(0, 0, cols, rows)
    band = band4 / band8
    # band[band > 1] = 9999  # 过滤异常值
    return band


#归一化差异绿度指数
def get_ndgi(path):  # 计算某一影像的ndvi值，返回二维数组
    dataset = gdal.Open(path)
    cols = dataset.RasterXSize  # 列数
    rows = dataset.RasterYSize  # 行数
    band4 = dataset.GetRasterBand(4).ReadAsArray(0, 0, cols, rows)  #红
    band3 = dataset.GetRasterBand(3).ReadAsArray(0, 0, cols, rows)  #绿
    molecule = band3 - band4
    denominator = band4 + band3
    del dataset
    band = molecule / denominator
    band[band > 1] = 9999  # 过滤异常值
    return band


def compute_band(file):
    dataset = gdal.Open(file)
    cols = dataset.RasterXSize  # 列数
    rows = dataset.RasterYSize  # 行数
    # 生成影像
    target_ds = gdal.GetDriverByName('GTiff').Create(result_path, xsize=cols, ysize=rows, bands=1,
                                                     eType=gdal.GDT_Float32)
    target_ds.SetGeoTransform(dataset.GetGeoTransform())
    target_ds.SetProjection(dataset.GetProjection())
    del dataset
    # band = get_ndvi(file)#NDVI
    # band = get_rvi(file)#RVI
    band = get_ndgi(file)  #NDGI

    target_ds.GetRasterBand(1).SetNoDataValue(9999)
    target_ds.GetRasterBand(1).WriteArray(band)
    target_ds.FlushCache()

In [8]:
import os

rootpath = os.listdir(r'E:\Desktop\Summary\种植结构提取-水稻\whole_datas')
for itme in rootpath:
    if itme.lower().endswith('.tif'):
        rater_file = os.path.join(r'E:\Desktop\Summary\种植结构提取-水稻\whole_datas', itme)
        result_path = os.path.join(r'E:\Desktop\Summary\种植结构提取-水稻\whole_datas\NDGI', 'NDGI' + itme)
        compute_band(rater_file)

C:\Users\dell\AppData\Local\Temp\ipykernel_18024\3827634227.py:42: RuntimeWarning: invalid value encountered in divide
  band = molecule / denominator


In [ ]:
#20230514
import os

rootpath = os.listdir(r'E:\Desktop\Summary\种植结构提取-水稻\datas\CLIP')
for itme in rootpath:
    rater_file = os.path.join(r'E:\Desktop\Summary\种植结构提取-水稻\datas\CLIP', itme)
    result_path = os.path.join(r'E:\Desktop\Summary\种植结构提取-水稻\datas\NDGI', 'NDGI' + itme[4:])
    compute_band(rater_file)